## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [ ]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [11]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [12]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [13]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [32]:
# prompt 를 PromptTemplate 객체로 생성합니다.
# prompt = PromptTemplate.from_template("{topic} 에 대해 {how} 설명해주세요.")
prompt = PromptTemplate.from_template("{topic} 에 대해 재미있게 설명해주세요.")

model = ChatOpenAI(model="gpt-4.1-nano", temperature=0.1)

chain = prompt | model

In [33]:
chain

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='{topic} 에 대해 재미있게 설명해주세요.')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002551F4EA010>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000255222B4750>, root_client=<openai.OpenAI object at 0x000002551F38DC10>, root_async_client=<openai.AsyncOpenAI object at 0x000002551EE1C750>, model_name='gpt-4.1-nano', temperature=0.1, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [29]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
# input = {"topic": "인공지능 모델의 학습 원리", "how": "전문가스럽게"}
input = {"topic": "인공지능 모델의 학습 원리"}

In [23]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

AIMessage(content='인공지능 모델의 학습 원리는 데이터를 이용해 모델이 문제를 해결하는 방법을 배우는 과정입니다. 일반적으로 다음과 같은 단계로 이루어집니다:\n\n1. **데이터 수집:** 모델이 학습할 수 있도록 많은 예제와 정보를 모읍니다.\n2. **모델 초기화:** 처음에는 무작위로 또는 기본값으로 모델을 만듭니다.\n3. **학습 과정:** 모델이 예제 데이터를 보고 정답과 비교하며, 예측이 얼마나 맞았는지 평가합니다. 이때 오차(실제 값과 예측 값의 차이)를 계산합니다.\n4. **파라미터 조정:** 오차를 줄이기 위해 모델 내부의 가중치(파라미터)를 수정합니다. 이 과정을 반복하면서 모델이 점점 더 정확하게 예측할 수 있도록 학습합니다.\n5. **검증 및 테스트:** 학습이 끝난 후, 새로운 데이터로 모델이 잘 작동하는지 확인합니다.\n\n이 과정을 통해 인공지능은 데이터를 바탕으로 패턴을 학습하고, 새로운 문제에 대해 더 잘 대응할 수 있게 됩니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 248, 'prompt_tokens': 24, 'total_tokens': 272, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_f0bc439dc3', 'id': 'chatcmpl-Co1JYSphi0lTKrat2ue2YkBPe36VP', 'service_tier': 'default

In [34]:
### 변수가 하나일 경우에는 예외적으로 딕셔너리 형태가 아니여도 가능
chain.invoke('나라별 안녕하세요를 설명해주세요')

AIMessage(content='물론이죠! "안녕하세요"는 한국에서 가장 흔하게 쓰이는 인사말로, 전 세계 어디를 가든지 "Hello" 또는 "Hi"와 비슷한 역할을 합니다. 그런데 나라별로 인사하는 방식이 얼마나 다양한지 아세요? 한번 재미있게 살펴볼까요!\n\n1. **한국 - "안녕하세요"**  \n   한국에서는 정중함이 중요해서, 누구든지 처음 만났을 때 "안녕하세요"라고 인사합니다. 마치 마법처럼 부드럽고 친근한 느낌을 주죠. 특히 나이와 관계에 따라 조금씩 변하는 인사말도 있는데요, "안녕"보다 더 공손한 "안녕하세요"가 바로 그 예입니다.\n\n2. **일본 - "こんにちは" (Konnichiwa)**  \n   일본에서는 낮 시간에 "こんにちは"라고 인사하는데, 이 말은 "좋은 하루"라는 의미를 담고 있어요. 일본인들은 인사할 때도 예의와 겸손을 중요시해서, 가볍게 "요"라고 끝내는 것이 특징입니다.\n\n3. **중국 - "你好" (Nǐ hǎo)**  \n   중국에서는 "你好"라고 말하는데, 이 말은 "너 좋다"라는 의미로, 상대방에게 친근함을 전하는 말이죠. 중국인들은 인사할 때도 손을 잡거나, 가볍게 고개를 숙이기도 합니다.\n\n4. **영국 - "Hello" 또는 "Hi"**  \n   영국에서는 "Hello"가 기본 인사이고, 좀 더 친근하게는 "Hi"라고 하죠. 때로는 "Good morning"이나 "Good afternoon"처럼 시간대별 인사도 사용합니다.\n\n5. **미국 - "Hi" 또는 "Hey"**  \n   미국에서는 "Hi"가 일상적이고, 친구끼리 "Hey"라고 부르기도 해요. 좀 더 격식을 차리고 싶을 때는 "Hello"를 사용하죠.\n\n6. **프랑스 - "Bonjour"**  \n   프랑스에서는 낮에는 "Bonjour"라고 인사하는데, 이 말은 "좋은 날"이라는 의미입니다. 저녁에는 "Bonsoir"라고 하죠.\n\n7. **아프리카 - 다양한 인사법**  \n   아프리카는 대륙이 넓어서 나라별로 인사

아래는 스트리밍을 출력하는 예시 입니다.

In [37]:
# 스트리밍 출력을 위한 요청
# answer = chain.stream(input)
answer = chain.stream('나라별 안녕하세요를 설명해주세요 한글로 어떻게 발음하는지도 함께요')
# 스트리밍 출력
stream_response(answer)

안녕하세요! 나라별로 "안녕하세요"라는 인사말이 어떻게 다르게 들리고, 어떻게 발음하는지 재미있게 살펴볼게요. 준비되셨나요? 출발!

1. 대한민국 (한국)
- 인사말: 안녕하세요
- 발음: [annyeonghaseyo]
- 설명: 한국에서는 "안녕하세요"라고 말하면, 마치 친구에게 따뜻한 포옹을 하는 것처럼 친근하고 정중한 인사예요. "안녕"은 "안녕히"의 줄임말이고, "하세요"는 존댓말이죠. 그래서 한국 사람들은 이 인사로 서로의 안부를 묻고, 마음을 전한답니다.

2. 일본
- 인사말: こんにちは (Konnichiwa)
- 발음: [konnichiwa]
- 설명: 일본에서는 "こんにちは"라고 해요. 한국의 "안녕하세요"와 비슷하게 낮과 낮은 인사이지만, 일본어는 좀 더 부드럽고 정중한 느낌이죠. "콘니치와"라고 발음하면, 일본 사람들도 친근하게 받아들여요.

3. 중국
- 인사말: 你好 (Nǐ hǎo)
- 발음: [ní hǎo]
- 설명: 중국에서는 "你好"라고 해요. "니 하오"라고 발음하는데, 이건 "너"와 "좋다"라는 의미를 담고 있어요. 그래서 중국 사람들은 "니 하오"라고 말하면, "안녕, 좋은 하루 보내!"라는 따뜻한 인사를 전하는 거죠.

4. 미국 (영어권)
- 인사말: Hello
- 발음: [həˈloʊ]
- 설명: 미국에서는 "Hello"라고 해요. 발음은 "헬로우"인데, 이건 정말 간단하고 친근한 인사죠. 미국 사람들은 "Hello"로 시작해서 친구, 가족, 낯선 사람 모두에게 인사를 건넵니다.

5. 프랑스
- 인사말: Bonjour
- 발음: [bɔ̃ʒuʁ]
- 설명: 프랑스에서는 "Bonjour"라고 해요. "봉쥬르"라고 발음하는데, 이건 "좋은 아침" 또는 "좋은 하루"라는 의미를 담고 있어요. 프랑스 사람들은 이 인사로 하루를 시작하죠.

6. 스페인
- 인사말: ¡Hola!
- 발음: [ˈola]
- 설명: 스페인에서는 "¡Hola!"라고 해요. "올라"라고 발음하는데, 이건 정말 친근하고 활기찬 인사예요. 스페인 사람들은 

### 출력파서(Output Parser)


In [38]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.

In [42]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

"물론이죠! 인공지능 모델의 학습 원리를 재미있게 설명해볼게요.\n\n상상해보세요, 인공지능은 마치 어린 아이가 새로운 것을 배우는 것과 비슷해요. 예를 들어, 아이가 고양이와 강아지를 구별하는 법을 배우는 과정이라고 생각하면 이해가 쉽죠.\n\n1. **데이터라는 '책'을 읽어요**  \n인공지능은 엄청난 양의 사진, 글, 소리 같은 데이터를 '읽으며' 배워요. 이 데이터들은 마치 아이가 여러 책을 읽으며 세상을 배우는 것과 같죠.\n\n2. **정답을 알려주는 '선생님'이 있어요**  \n이때, 인공지능에게는 정답이 적힌 '답안지' 같은 것이 필요해요. 예를 들어, 어떤 사진이 고양이인지 강아지인지 알려주는 정답이죠. 이 정답을 보고 인공지능은 '아, 이게 고양이구나!' 하고 배워가는 거예요.\n\n3. **실수하고 고치는 과정, '연습'이에요**  \n처음에는 인공지능이 틀릴 수도 있어요. 마치 어린 아이가 처음 그림을 그릴 때처럼요. 그래서 인공지능은 틀린 점을 찾아내고, 더 잘 맞도록 계속 수정하면서 배우는 거예요. 이 과정을 '학습'이라고 부르죠.\n\n4. **경험을 쌓으며 똑똑해져요**  \n많은 데이터를 보고, 계속 실수하고 고치면서 인공지능은 점점 더 똑똑해지고, 새로운 사진이나 글도 잘 구별할 수 있게 되는 거예요.\n\n요약하자면, 인공지능은 엄청난 양의 데이터를 읽고, 정답을 참고하며, 실수에서 배우는 '지혜로운 학생'과 같다고 할 수 있어요. 그래서 우리가 인공지능에게 많은 데이터를 보여주면 보여줄수록, 더 똑똑해지고 멋진 일을 할 수 있게 되는 거죠!\n\n혹시 더 궁금한 점 있나요?"

In [43]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

AIMessage(content="물론이죠! 인공지능 모델의 학습 원리를 재미있게 설명해볼게요.\n\n생각해보세요, 인공지능은 마치 아주 똑똑한 학생이 시험을 보는 것과 비슷해요. 이 학생은 처음에는 아무것도 몰라서 여러 문제를 풀 때마다 틀리기도 하고 맞기도 하죠. 그런데 이 학생이 계속해서 공부하고 연습하면서 점점 더 똑똑해지는 것처럼, 인공지능도 '학습'이라는 과정을 통해 점점 더 똑똑해지는 거예요.\n\n이 과정은 이렇게 진행돼요:\n\n1. **데이터라는 교과서**: 인공지능에게는 엄청 많은 사진, 글, 소리 같은 '자료'가 필요해요. 이 자료들이 바로 교과서 같은 역할을 하죠.\n\n2. **모델이라는 학생**: 인공지능은 '모델'이라는 이름의 학생이에요. 이 학생은 처음에는 아무것도 몰라서 답을 잘 못 맞춰요.\n\n3. **학습이라는 공부**: 학생이 시험을 보고 답이 틀리면, 선생님이 '이건 잘못됐어요!'라고 알려주듯이, 인공지능도 정답과 비교해서 틀린 부분을 찾아내요. 그리고 '이 부분을 이렇게 수정하면 더 좋아질 거야'라고 배우는 거죠.\n\n4. **반복 연습**: 이 과정을 수천, 수만 번 반복하면서 학생은 점점 더 정답에 가까워지고, 결국에는 거의 완벽하게 문제를 풀 수 있게 돼요.\n\n5. **보상과 벌**: 재미있는 점은, 인공지능은 '보상'이라는 개념도 배워요. 정답에 가까워질수록 '좋았어!'라고 칭찬받는 것처럼, 더 잘하는 방향으로 계속 수정하는 거죠.\n\n이렇게 해서 인공지능은 많은 데이터를 통해 '패턴'을 배우고, 새로운 문제도 척척 해결할 수 있게 되는 거랍니다. 마치 공부를 열심히 해서 어느새 전문가가 되는 것과 비슷하죠!\n\n요약하자면, 인공지능의 학습은 '많은 자료를 보고, 틀린 점을 고치며, 반복해서 연습하는 것'이랍니다. 그래서 점점 더 똑똑해지고, 우리 일상 속 다양한 문제를 해결하는 멋진 도구가 되는 거예요!", additional_kwargs={'refusal': None}, response_metada

In [41]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

물론이죠! 인공지능 모델의 학습 원리를 재미있게 설명해볼게요.

생각해보세요, 인공지능은 마치 어린 아이가 새로운 것을 배우는 것과 비슷해요. 예를 들어, 그림을 보고 "이건 고양이야"라고 가르쳐주는 것처럼요. 그런데 이 아이는 처음에는 아무것도 몰라서, 수많은 그림을 보여주면서 "이게 고양이야" 또는 "이게 강아지야"라고 반복해서 알려줍니다.

이 과정에서 인공지능은 '데이터'라는 그림과 정답(고양이 또는 강아지)을 보고, 그 패턴을 찾기 시작해요. 마치 퍼즐 조각을 맞추듯이, 그림 속 특징(귀 모양, 눈 위치, 털 색깔 등)을 분석해서 "아, 이 특징들이 모이면 고양이구나!"라고 배워가는 거죠.

이때 사용하는 것이 바로 '학습 알고리즘'이라는 마법 같은 규칙인데요, 이 규칙은 인공지능이 틀린 답을 했을 때 "아, 이게 잘못됐구나"라고 알려주고, 다시 수정하는 과정을 반복하면서 점점 더 똑똑해집니다. 이 과정을 '훈련'이라고 부르죠.

쉽게 말해, 인공지능은 수많은 그림과 정답을 보고, "이 특징들이 모이면 고양이구나!"라는 규칙을 스스로 찾아내는 '머리 좋은 학생'과 같다고 할 수 있어요. 그리고 계속해서 연습하고 실수하면서 점점 더 정확하게 그림을 구별할 수 있게 되는 거죠.

그래서 인공지능이 멋지게 학습하는 비밀은 바로 '많은 데이터와 반복 학습'에 숨어 있답니다! 재미있지 않나요?

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [ ]:
template_ex = """
{{question}} escape 처리
"""


In [ ]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
대답의 인물이 다를 경우 A: ,B: 처럼 표기해주세요
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4.1-nano")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [48]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [49]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:
A: Hello, I’d like to order some food, please.  
B: Sure! What would you like to have?  
A: I’ll have the spaghetti and a glass of water.  
B: Thank you. Anything else?  
A: No, that’s all for now.  
B: Alright. Your order will be ready shortly.  

- 한글 해석:
A: 안녕하세요, 음식을 주문하고 싶어요.  
B: 네! 무엇을 드시겠어요?  
A: 스파게티와 물 한 잔 주세요.  
B: 감사합니다. 더 필요한 것은 없나요?  
A: 아니요, 그게 다예요.  
B: 알겠습니다. 곧 주문하신 음식이 준비될 거예요.


In [50]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream("저는 식당에 가서 음식을 주문하고 싶어요")
# 스트리밍 출력
stream_response(answer)

- 영어 회화:
A: Hello! I would like to order a meal, please.
B: Sure! What would you like to have?
A: I’ll have the grilled salmon with rice and a glass of water.
B: Certainly. Anything else?
A: No, that’s all, thank you.
- 한글 해석:
A: 안녕하세요! 주문하고 싶어요.
B: 네! 무엇을 드시겠어요?
A: 연어구이와 밥, 그리고 물 한 잔 주세요.
B: 알겠습니다. 더 필요하신 것은 없어요?
A: 아니요, 그것으로 주세요, 감사합니다.

In [51]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:
A: Hi! I’d like to order a large pepperoni pizza, please.  
B: Sure! Would you like any extra toppings or drinks with that?  
A: Yes, I’ll also get a side of garlic knots and a two-liter soda.  
B: Great! Your order will be ready in about 20 minutes.  
A: Thank you! How much is it in total?  
B: It’s $25.50. Would you like to pay now or when you pick it up?  
A: I’ll pay now. Here you go.  
B: Thank you! Your order is confirmed. See you soon!  

- 한글 해석:  
A: 안녕하세요! 큰 페퍼로니 피자 하나 주문할게요.  
B: 네! 추가 토핑이나 음료도 같이 주문하시겠어요?  
A: 네, 마늘빵도 하나 주시고, 두 리터 소다도 같이 할게요.  
B: 좋아요! 주문하신 피자는 약 20분 후에 준비됩니다.  
A: 감사합니다! 총 금액이 얼마인가요?  
B: $25.50입니다. 지금 결제하실 건가요, 아니면 픽업하실 때 결제하실 건가요?  
A: 지금 결제할게요. 여기 있습니다.  
B: 감사합니다! 주문이 확인되었습니다. 곧 뵙겠습니다!